In [70]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from functools import reduce
from api_keys import geoapify_key
import json
import requests
import warnings
warnings.filterwarnings('ignore')

In [86]:
dataset_2016 = 'Burlington_StatsCanada_2016_Census.csv'
dataset_2021 = 'Burlington_StatsCanada_2021_Census.csv'
private_schools = 'Private School Boards Ontario.csv'
public_schools = 'Public School Boards Ontario.csv'
house_sale_median = 'Burlington_Median_Sale_Price.csv'
house_rent_median = 'Burlington_Median_Rental_Price.csv'
house_sale_average = 'Burlington_Avg_Sale_Price.csv'
house_rent_average = 'Burlington_Avg_Rental_Price.csv'
new_builds_2016 = 'NewBuillds_Burlington_2016.csv'
new_builds_2021 = 'New_Builds_Burlington_2021.csv'
property_tax_2016 = '2016-Final-Tax-Rates.csv'
property_tax_2021 = '2021_Final_Tax_Rates.csv'
df_2016 = pd.read_csv(dataset_2016, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_2021 = pd.read_csv(dataset_2021, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_private_schools = pd.read_csv(private_schools, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_public_schools = pd.read_csv(public_schools, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_sale_median = pd.read_csv(house_sale_median, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_rent_median = pd.read_csv(house_rent_median, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_sale_average = pd.read_csv(house_sale_average, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_rent_average = pd.read_csv(house_rent_average, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_new_builds_2021 = pd.read_csv(new_builds_2021, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_new_builds_2016 = pd.read_csv(new_builds_2016, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_2016_tax = pd.read_csv(property_tax_2016, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_2021_tax = pd.read_csv(property_tax_2021, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')

,ï»¿Board Number,Board Name,Board Type,School Number,School Name,School Type,School Special Condition Code,School Level,School Language,Grade Range,...,Change in Grade 6 Writing Achievement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Mathematics,Change in Grade 6 Mathematics Achievement Over Three Years,Percentage of Grade 9 Students Achieving the Provincial Standard in Mathematics,Change in Grade 9 Mathematics Achievement Over Three Years,Percentage of Students That Passed the Grade 10 OSSLT on Their First Attempt,Change in Grade 10 OSSLT Literacy Achievement Over Three Years,Percentage of School-Aged Children Who Live in Low-Income Households,"Percentage of Students Whose Parents Have No Degree, Diploma or Certificate",Extract Date
2228,B29009,Northeastern CDSB,Cath Dist Sch Brd (E/F),689491,Bishop Belleau School,Catholic,Not applicable,Elementary,English,JK-8,...,NaN,0%,NaN,NaN,NaN,NaN,NaN,40,10,3-Nov-23
3497,B67059,Toronto CDSB,Cath Dist Sch Brd (E/F),832405,St Maurice Catholic School,Catholic,Not applicable,Elementary,English,JK-8,...,NaN,32%,NaN,NaN,NaN,NaN,NaN,45,20,3-Nov-23
3854,B66052,Toronto DSB,Pub Dist Sch Brd (E/F),382068,Karen Kain School of the Arts,Public,Not applicable,Elementary,English,6-8,...,NaN,39%,NaN,NaN,NaN,NaN,NaN,10,0,3-Nov-23


In [114]:
housing_dfs = [df_sale_median, df_rent_median, df_sale_average, df_rent_average]
df_housing_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Year'], how='outer'), housing_dfs)
years_to_keep = [2016, 2021]
df_housing_merged_filtered = df_housing_merged[df_housing_merged['Year'].isin(years_to_keep)]
renamed_columns = {
    'Sale Price, Median': 'Median Sale Price ($)',
    'Original Price, Median': 'Median Rent ($)',
    'Sale Price, Average': 'Average Sale Price ($)',
    'Original Price, Average': 'Average Rent ($)'
}
df_housing_merged_filtered.rename(columns=renamed_columns, inplace=True)

C:\Users\qwert\AppData\Local\Temp\ipykernel_20776\1609460120.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_housing_merged_filtered.rename(columns=renamed_columns, inplace=True)


,Year,Median Sale Price ($),Median Rent ($),Average Sale Price ($),Average Rent ($)
0,2016,609000,2400,663675,2710
5,2021,975500,2800,1081961,2862


In [41]:
private_school_count = len(df_private_schools.loc[df_private_schools['City'] == 'Burlington'])


2


In [42]:
public_school_count = len(df_public_schools.loc[(df_public_schools['City'] == 'Burlington') & (df_public_schools['School Type'] == 'Public')])
catholic_school_count = len(df_public_schools.loc[(df_public_schools['City'] == 'Burlington') & (df_public_schools['School Type'] == 'Catholic')])


38
17


In [79]:
new_builds_2016_count = df_new_builds_2016['All'].dropna().sum()
new_builds_2021_count = df_new_builds_2021['All'].dropna().sum()


542.0
1494.0


In [95]:
property_tax_rate_2021 = df_2021_tax['Total Urban Area'][0]
property_tax_rate_2016 = df_2016_tax['Total Urban Area'][0]


0.00779583 0.00871013


In [44]:
target_city = 'Burlington, ON, Canada'

qp = {
    "text": target_city,
    "apiKey": geoapify_key
}
base_url = "https://api.geoapify.com/v2/places"
geocode_url = 'https://api.geoapify.com/v1/geocode/search'
target_url = f"https://api.geoapify.com/v1/geocode/search?text={target_city}&format=json&apiKey={geoapify_key}"
response = requests.get(geocode_url, params=qp).json()
from pprint import pprint




{'features': [{'bbox': [-79.9592676, 43.2862845, -79.713939, 43.4765533],
               'geometry': {'coordinates': [-79.7966835, 43.3248924],
                            'type': 'Point'},
               'properties': {'address_line1': 'Burlington, ON',
                              'address_line2': 'Canada',
                              'category': 'administrative',
                              'city': 'Burlington',
                              'country': 'Canada',
                              'country_code': 'ca',
                              'county': 'Halton Region',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                             'license': 'Open Database License',
                                             'sourcename': 'openstreetmap',
                                             'url': 'https://www.openstreetmap.org/copyright'},
        

In [45]:
lat = response['features'][0]['properties']['lat']
lon = response['features'][0]['properties']['lon']



43.3248924 -79.7966835


In [27]:
#for index, row in leisure_df.iterrows():

#print(leisure_df)
categories = 'leisure'
radius = 183000
limit=300
params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": limit,
    "radius": radius
}
params["filter"] = f'circle:{lon},{lat},{radius}'
params["bias"] = f'proximity:{lon},{lat}'
leisure_dict = requests.get(base_url, params=params).json()

leisure_types = []
for feature in leisure_dict['features']:
    leisure_info = feature['properties']['datasource']['raw']['osm_id']
    if 'city' in feature['properties']:
        leisure_cities = feature['properties']['city']
        if leisure_cities == 'Burlington': 
            leisure_types.append(leisure_info)
    
leisure_count = len(leisure_types)


    

282


In [108]:
def format_value(value, percentage=False, currency=False):
    if isinstance(value, np.ndarray):
        value = value.item()
    if percentage:
        return f"{value:.2f}%"
    elif currency:
        return f"${value:.2f}"
    else:
        return str(round(value))
    
pop_2016 = df_2016.loc[df_2016['Characteristics'] == 'Population; 2016']['Total'].values
income_2016 = df_2016.loc[df_2016['Characteristics'] == '    Average total income in 2015 among recipients ($)']['Total'].values
house_value_2016 = df_2016.loc[df_2016['Characteristics'] == '  Average value of dwellings ($)']['Total'].values
unemployment_2016 = df_2016.loc[df_2016['Characteristics'] == 'Unemployment rate']['Total'].values
rented_2016 = df_2016.loc[df_2016['Characteristics'] == '  Renter']['Total'].values
owned_2016 = df_2016.loc[df_2016['Characteristics'] == '  Owner']['Total'].values
burlington_2016 = {
    'City': 'Burlington',
    'Year': '2016',
    'Population': format_value(pop_2016),
    'Household Income ($)': format_value(income_2016, currency = True),
    'Estimated House Value ($)': format_value(house_value_2016, currency = True),
    'Unemployment Rate (%)': format_value(unemployment_2016, percentage = True),
    'Rented Houses': format_value(rented_2016),
    'Owned Houses': format_value(owned_2016),
    'New Built': format_value(new_builds_2016_count),
    'Property Tax Rate (%)': property_tax_rate_2016
}
burlington_2016_df_formatted = pd.DataFrame(burlington_2016, index = [0])


   Year Population Household Income House Values Unemployment Rate  \
0  2016     183314        $60898.00   $632556.00             5.60%   

  Rented Housing Owned Housing New Builds  Property Tax Rate  
0          16835         54540        542            0.00871  


In [107]:
pop_2021 = df_2021.loc[df_2021['Characteristic'] == 'Population, 2021']['Total'].values
income_2021 = df_2021.loc[df_2021['Characteristic'] == '    Average total income in 2019 among recipients ($)']['Total'].values
house_value_2021 = df_2021.loc[df_2021['Characteristic'] == '  Average value of dwellings ($)']['Total'].values
unemployment_2021 = df_2021.loc[df_2021['Characteristic'] == 'Unemployment rate']['Total'].values
rented_2021 = df_2021.loc[df_2021['Characteristic'] == '  Renter']['Total'].values
owned_2021 = df_2021.loc[df_2021['Characteristic'] == '  Owner']['Total'].values



burlington_2021 = {
    'Year': '2021',
    'City': 'Burlington',
    'Population': format_value(pop_2021),
    'Household Income ($)': format_value(income_2021, currency = True),
    'Estimated House Value ($)': format_value(house_value_2021, currency = True),
    'Unemployment Rate (%)': format_value(unemployment_2021, percentage = True),
    'Rented Houses': format_value(rented_2021),
    'Owned Houses': format_value(owned_2021),
    'New Built': format_value(new_builds_2021_count),
    'Property Tax Rate (%)': property_tax_rate_2021
}
burlington_2021_df_formatted = pd.DataFrame(burlington_2021, index = [0])


   Year Population Household Income House Values Unemployment Rate  \
0  2021     186948        $67200.00  $1024000.00            10.70%   

  Rented Housing Owned Housing New Builds  Property Tax Rate  
0          18220         54955       1494           0.007796  


In [128]:
amenities = {
    'City': 'Burlington',
    '# of Public Schools': public_school_count,
    '# of Catholic Schools': catholic_school_count,
    '# of Private Schools': private_school_count,
    '# of Leisure Spaces': leisure_count
}

amenities_df = pd.DataFrame(amenities, index = [0])
amenities_df.set_index('City', inplace=True)
amenities_df

,# of Public Schools,# of Catholic Schools,# of Private Schools,# of Leisure Spaces
City,,,,
Burlington,38,17,2,282


In [1]:
combined_df_stats_formatted = pd.concat([burlington_2016_df_formatted, burlington_2021_df_formatted], ignore_index=True)
combined_df_stats_formatted.set_index(['Year'])

combined_df_stats_formatted['Year'] = combined_df_stats_formatted['Year'].astype(int)
df_housing_merged_filtered['Year'] = df_housing_merged_filtered['Year'].astype(int)

combined_df = pd.merge(combined_df_stats_formatted, df_housing_merged_filtered, on='Year')


NameError: name 'pd' is not defined

In [126]:
pop_2016 = df_2016.loc[df_2016['Characteristics'] == 'Population; 2016']['Total'].values
income_2016 = df_2016.loc[df_2016['Characteristics'] == '    Average total income in 2015 among recipients ($)']['Total'].values
house_value_2016 = df_2016.loc[df_2016['Characteristics'] == '  Average value of dwellings ($)']['Total'].values
unemployment_2016 = df_2016.loc[df_2016['Characteristics'] == 'Unemployment rate']['Total'].values
rented_2016 = df_2016.loc[df_2016['Characteristics'] == '  Renter']['Total'].values
owned_2016 = df_2016.loc[df_2016['Characteristics'] == '  Owner']['Total'].values
burlington_2016 = {
    'Year': '2016',
    'City': 'Burlington',
    'Population': pop_2016,
    'Household Income ($)': income_2016,
    'Estimated House Value ($)': house_value_2016,
    'Unemployment Rate (%)': unemployment_2016,
    'Rented Houses': rented_2016,
    'Owned Houses': owned_2016,
    'New Built': new_builds_2021_count,
    'Property Tax Rate (%)': property_tax_rate_2021
}
burlington_2016_df = pd.DataFrame(burlington_2016, index = [0])

pop_2021 = df_2021.loc[df_2021['Characteristic'] == 'Population, 2021']['Total'].values
income_2021 = df_2021.loc[df_2021['Characteristic'] == '    Average total income in 2019 among recipients ($)']['Total'].values
house_value_2021 = df_2021.loc[df_2021['Characteristic'] == '  Average value of dwellings ($)']['Total'].values
unemployment_2021 = df_2021.loc[df_2021['Characteristic'] == 'Unemployment rate']['Total'].values
rented_2021 = df_2021.loc[df_2021['Characteristic'] == '  Renter']['Total'].values
owned_2021 = df_2021.loc[df_2021['Characteristic'] == '  Owner']['Total'].values



burlington_2021 = {
    'Year': '2021',
    'City': 'Burlington',
    'Population': pop_2021,
    'Household Income ($)': income_2021,
    'Estimated House Value ($)': house_value_2021,
    'Unemployment Rate (%)': unemployment_2021,
    'Rented Houses': rented_2021,
    'Owned Houses': owned_2021,
    'New Built': new_builds_2021_count,
    'Property Tax Rate (%)': property_tax_rate_2016
}
burlington_2021_df = pd.DataFrame(burlington_2021, index = [0])


combined_df_stats = pd.concat([burlington_2016_df, burlington_2021_df], ignore_index=True)
combined_df_stats.set_index(['Year'])
combined_df_stats['Year'] = combined_df_stats['Year'].astype(int)
df_housing_merged_filtered['Year'] = df_housing_merged_filtered['Year'].astype(int)

combined_df = pd.merge(combined_df_stats, df_housing_merged_filtered, on='Year')



columns_order = [
    'City', 'Year', 'Household Income ($)', 'Average Rent ($)', 'Median Rent ($)',
    'Estimated House Value ($)', 'Unemployment Rate (%)', 'Property Tax Rate (%)',
    'Rented Houses', 'Owned Houses', 'New Built', 'Average Sale Price ($)', 'Median Sale Price ($)'
]

combined_df = combined_df[columns_order]

combined_df.set_index(['City', 'Year'], inplace=True)




C:\Users\qwert\AppData\Local\Temp\ipykernel_20776\1824696828.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_housing_merged_filtered['Year'] = df_housing_merged_filtered['Year'].astype(int)


Household Income ($)  Average Rent ($)  Median Rent ($)  \
City       Year                                                            
Burlington 2016               60898.0              2710             2400   
           2021               67200.0              2862             2800   

                 Estimated House Value ($)  Unemployment Rate (%)  \
City       Year                                                     
Burlington 2016                   632556.0                    5.6   
           2021                  1024000.0                   10.7   

                 Property Tax Rate (%)  Rented Houses  Owned Houses  \
City       Year                                                       
Burlington 2016               0.007796        16835.0       54540.0   
           2021               0.008710        18220.0       54955.0   

                 New Built  Average Sale Price ($)  Median Sale Price ($)  
City       Year                                                            
Burlington 2016     1494.0                  663675                 609000  
           2021     1494.0                 1081961                 975500

In [2]:
combined_df
amenities_df

NameError: name 'combined_df' is not defined